# MNIST Tutorial : classify digits using Tensorflow

This tutorial is an introduction to tensorflow. It is inspired from the [beginners' TensorFlow tutorial](https://www.tensorflow.org/tutorials/keras/basic_classification). At the end, given an image of a digit, you will be able to predict which digit it is. You will also understand the principal steps to train a neural network model.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np  # library to process arrays
import matplotlib.pyplot as plt # to use plots
from itertools import islice, cycle
from tqdm import tqdm
import os


### Import data
MNIST is a simple computer vision dataset. It consists of images of handwritten digits.
It will be downloaded in your current directory (or loaded from the specified directory if you already downloaded it)

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

### MNIST Dataset structure
* training set (60 000) : `train_images`, `train_labels`
* testing set (10 000) : `test_images`, `test_labels`

In [ ]:
print('Shape of training images : ', train_images.shape)
print('Length of training labels : ', len(train_labels))

The images have dimensions `28x28` and their pixel values range from 0 to 255.

#### Example of one digit image

In [ ]:
# Select an image to plot
index = 43

plt.figure()
plt.imshow(train_images[index], cmap='gray')
plt.colorbar()
plt.grid(False)
plt.title('label = {}'.format(train_labels[index]))

Labels is an array containing the corresponding class (0 to 9) for each sample of the set. So the nth element of `labels` is the class of the nth element of `images` 

In [ ]:
train_labels

______
### Preprocessing

The pixels values are scaled between 0 and 1 before feeding to the neural network model. We call this step normalization.
As seen in the previous tutorial the maximum value of a pixel in 8-bits representation is 255 (2^8) so we normalize all the pixels values by 255.

WARNING : It's important that you preporcess youre training and testing data the same way.

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])

_______
### Some key TF concepts 

#### Tensor
The central unit of data in TensorFlow is the tensor. A tensor consists of a set of primitive values shaped into an array of any number of dimensions. A tensor's rank is its number of dimensions.

#### [Tensorflow's computational graph](https://www.tensorflow.org/guide/graphs)
A computational graph is a series of TensorFlow operations arranged into a graph of nodes. Each node takes zero or more tensors as inputs and produces a tensor as an output.
There are two separate steps in a Tensorflow programm : 
* Building the graph
* Running the graph

To evaluate the nodes, the computational graph must be run within a session. A session encapsulates the control and state of the TensorFlow runtime.
______

### Simple model
We will build a simple neuron that take an input `x`, multiplies it by some weight `W` and adds a bias `b`.

In [ ]:
# Create the neuron
x = tf.placeholder(tf.float32, [None, 28, 28], name='input_image')
W = tf.Variable(tf.zeros([784, 10]), name='weights')
b = tf.Variable(tf.zeros([10]), name='bias')
x_flatten = tf.layers.flatten(x)
y = tf.matmul(x_flatten, W) + b

`x` isn't a specific value. It's a `placeholder`, a value that we'll input when we ask TensorFlow to run a computation. We want to be able to input any number of MNIST images, we represent this as a 2D tensor of floating-point numbers (`tf.float32`), with a shape `[None, 28, 28]`. (`None` means that a dimension can be of any length.)

We also need the weights and biases for our model. A `Variable` is a modifiable tensor that lives in TensorFlow's graph of interacting operations. It can be used and even modified by the computation. For machine learning applications, one generally has the model parameters be `Variable`s.
`W` and `b` are initialized with zeros (`tf.zeros`).
Notice that `W` has a shape of `[784, 10]` because we want to multiply the 784-dimensional image vectors by it to produce 10-dimensional vectors of evidence for the difference classes. `b` has a shape of `[10]` so we can add it to the output.
Then we multiply (`tf.matmul`) the input `x` by the weights `W` and add the bias `b`! (`y = x*W + b`)

#### Softmax regression
Every image in MNIST is of a handwritten digit between zero and nine. So there are only ten possible things that a given image can be. We want to be able to look at an image and give the probabilities for it being each digit.

This is a classic case where a softmax regression is a natural, simple model. If you want to assign probabilities to an object being one of several different things, softmax is the thing to do, because softmax gives a list of values between 0 and 1 that add up to 1. Even later on, when we train more sophisticated models, the final step will be a layer of softmax.

In [ ]:
%%latex
$\text{softmax}(x)_i = \frac{\exp(x_i)}{\sum_j \exp(x_j)}$
with $i$ the class and $j$ the pixel index

In [ ]:
# Apply softmax 
y_softmax = tf.nn.softmax(y, name='softmax')

The model is now defined (yes, in one line!)
____

### Training the model
In order to train our model, we need to define what it means for the model to be good. Well, actually, in machine learning we typically define the oposite, i.e what it means for a model to be bad. This is called the cost, or the loss, and it represents how far off the model is from the desired outcome. We try to minimize that error, and the smaller the error margin, the better the model is.

One very common, very nice function to determine the loss of a model is called [`cross-entropy`](https://en.wikipedia.org/wiki/Cross_entropy).

In [ ]:
%%latex
$ crossentropy(y, y') = -\sum y'\log(y)$
with $y$ the predicted probability distribution and $y'$ the true distribution

#### Loss

In [ ]:
y_ = tf.placeholder(tf.int32, [None], 'labels')
y_one_hot = tf.one_hot(y_, depth=10)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_one_hot * tf.log(y_softmax), axis=1), name='crosscentropy_loss')

First we add a new placeholder `y_` to input the correct answers (i.e the true label).
A one-hot vector is a vector which is `0` in most dimensions, and `1` in a single dimension. In our case, the nth digit will be represented as a vector which is `1` in the nth dimension.

Then we implement the cross-entropy function, with `tf.log` computing the logarithm of each element of `y_softmax`, then each element of `y_`is multiplied by the corresponding element of `tf.log(y_softmax)`. Then `tf.reduce_sum` adds the elements in the second dimension of `y_softmax` (`axis=1` parameter). Finally, `tf.reduce_mean` computes the mean over all the examples in the batch.

Because this implementation is numerically instable, we use instead Tensorflow built-in function `tf.nn.softmax_cross_entropy_with_logits_v2` on the unnormalized logits (i.e `y`). This function internally computes the softmax activation.

In [ ]:
# So with the built-in tf function we get
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_one_hot, logits=y))

#### Loss minimization
Because TensorFlow knows the entire graph of your computations, it can automatically use the [backpropagation algorithm](https://colah.github.io/posts/2015-08-Backprop) to efficiently determine how your `Variable`s affect the loss you ask it to minimize. Then it can apply your choice of optimization algorithm to modify the variables and reduce the loss.

In our case, we will ask TensorFlow to minimize `cross_entropy` using the [gradient descent algorithm](https://en.wikipedia.org/wiki/Gradient_descent) with a learning rate of 0.5. Gradient descent is a simple procedure, where TensorFlow simply shifts each variable a little bit in the direction that reduces the cost. 

If you want to try another optimization algorithm just pick one from [these](https://www.tensorflow.org/versions/r1.0/api_guides/python/train#Optimizers)

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

#### Launching a session
We use an `tf.InteractiveSession` (it would be also possible to start a `tf.Session`) and initialize the `Variable`s.

And then just start the training by running `train_step` 1000 times. 
At each step of the loop, we will get a "batch" (`batch_xs`, `batch_ys`) of `batch_size` random data points from our training set. We run `train_step` feeding in (via `feed_dict{}`) the batches data to replace the `placeholder`s.

Using small batches of random data is called stochastic training -- in this case, stochastic gradient descent. Ideally, we'd like to use all our data for every step of training because that would give us a better sense of what we should be doing, but that's expensive. So, instead, we use a different subset every time. Doing this is cheap and has much of the same benefit.

In [ ]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Generate an iterator to cycle through the training data
train_images_it = cycle(train_images)
train_labels_it = cycle(train_labels)

batch_size = 100
for _ in tqdm(range(1000)):
    batch_xs, batch_ys = (list(islice(train_images_it, batch_size)), list(islice(train_labels_it, batch_size)))
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

### Evaluating the model
We'd like to know how well our model performs, so we use for this the test dataset `mnist.test` (which should _never_ be seen during training).

We will compare the predicted digit with the label and compute some metrics to estimate the performance.

In [ ]:
with tf.name_scope('evaluation'):
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_one_hot,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict={x: test_images, y_: test_labels}))

`tf.argmax` returns the index of the highest entry in a tensor along some axis, so `tf.argmax(y,1)` returns the digit that has been predicted. `tf.equal` compare two elements and outputs `True` when they are equal and `False` otherwise.
The accuracy is the mean `tf.reduce_mean` of comparisons between the predicted digit `y` and its label `y_`. 
The accuracy should be around 91-92%.

## That's all folks, now you have a (simple) model!

### Tensorboard
To visualize the graph of your model you can run from a terminal `tensorboard --logdir logdir`

Then go to `localhost:6006`

__Warning__ execute each cell only once to have a nice graph

In [ ]:
writer = tf.summary.FileWriter(logdir=os.path.join('logdir', 'simple'), graph=tf.get_default_graph())

#### Want to try it ? Select an image of digit and feed it to the model

In [ ]:
# Chose an image to feed
index = 140
digit_image = test_images[index]
plt.imshow(test_images[index])

In [ ]:
prediction = tf.argmax(tf.nn.softmax(y), axis=1)
sess.run(prediction, feed_dict={x: digit_image[None]}) # Add None to have a array of shape [1, 784]

___
## If you want to build a bigger model, and have a better accuracy you can go further on the Notebook

In practice when we want to build more complex model with more layers we will use higher-level tensorflow functions, that will help to make the code more readable and will take care of several initializations and paramters for us.

The placeholders' definitions are quite the same as the simple network's ones.

In [ ]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, 28, 28], name='input')
y_ = tf.placeholder(tf.int32, [None], 'labels')
y_one_hot = tf.one_hot(y_, depth=10)

In [ ]:
model_name = 'dense'

Define two models, one using only fully connected layers (`Dense`), and one using convolutional layers (`Conv2D`)

### Dense Model
The first layer in this network, `tf.keras.layers.Flatten`, transforms the format of the images from a 2d-array (of 28 by 28 pixels), to a 1d-array of 28 * 28 = 784 pixels. Think of this layer as unstacking rows of pixels in the image and lining them up. This layer has no parameters to learn; it only reformats the data.

After the pixels are flattened, the network consists of a sequence of two `tf.keras.layers.Dense` layers. These are densely-connected, or fully-connected, neural layers. The first Dense layer has 512 nodes (or neurons). The second layer is a 10-node dense layer followed by a softmax layer—this returns an array of 10 probability scores that sum to 1. Each node contains a score that indicates the probability that the current image belongs to one of the 10 classes.

In [ ]:
if model_name == 'dense':
    with tf.name_scope('DenseModel'):
        net = keras.layers.Flatten()(x)
        net = keras.layers.Dense(512, activation=tf.nn.relu)(net)
        net = keras.layers.Dropout(0.2)(net)
        logprob = keras.layers.Dense(10)(net)
        y_pred = keras.layers.Activation('softmax')(logprob)
        
    batch_size = 256
    n_steps = 2000

### Convolutional model

__First  layer : convolution + max-pooling__ - 
The first layer will consist of a convolution, followed by max pooling. The convolution will compute 16 features for each 3x3 patch. 
Then we apply the ReLU activation and max-pool the result. This results in a tensor of shape `[?, 14, 14, 32]`

__Second layer : convolution + max-pooling__ - 
Repeat the layer. This time the convolution computes 32 features. The 2x2 max-pooling will reduce the size of the output to `[?, 7, 7, 64]`

__Third layer : densely connected__ - 
We reshape the tensor from the pooling layer into a batch of vectors and we add a fully-connected layer with 256 neurons to allow processing on the entire image.

__Last layer : readout__ - 
Last layer with 10 clasees output to apply the softmax function.

In [ ]:
if model_name == 'convolutional':
    with tf.name_scope('ConvolutionalModel'):
        net = keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation=tf.nn.relu)(tf.expand_dims(x, axis=-1))
        net = keras.layers.MaxPool2D()(net)
        net = keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.nn.relu)(net)
        net = keras.layers.MaxPool2D()(net)
        net = keras.layers.Flatten()(net)
        net = keras.layers.Dense(256, activation=tf.nn.relu)(net)
        logprob = keras.layers.Dense(10)(net)
        y_pred = keras.layers.Activation('softmax')(logprob)
        
    batch_size = 256
    n_steps = 4000

We use the same loss as the 'simple' model (perceptron)

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_one_hot, logits=logprob))

But we use another optimizer 

In [ ]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

Again, we run the tensorflow graph

__WARNING__ : the training might take 15-20 minutes and take all the ressources of your computer.

In [ ]:
session2 = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Generate an iterator to cycle through the training data
train_images_it = cycle(train_images)
train_labels_it = cycle(train_labels)

for _ in tqdm(range(n_steps)):
    batch_xs, batch_ys = (list(islice(train_images_it, batch_size)), list(islice(train_labels_it, batch_size)))
    session2.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

Evaluation of the complex model

In [ ]:
correct_prediction = tf.equal(tf.argmax(logprob,1), tf.argmax(y_one_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(session2.run(accuracy, feed_dict={x: test_images, y_: test_labels}))

## Tensorboard
To visualize the graph of your model you can run from a terminal `tensorboard --logdir logdir`

Then go to `localhost:6006`

__Warning__ execute each cell only once to have a nice graph

In [ ]:
writer = tf.summary.FileWriter(logdir=os.path.join('logdir', 'complex'), graph=tf.get_default_graph())